In [1]:
import os
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Dense,Dropout,Activation, Conv2D, MaxPool2D, Flatten
import numpy as np
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras import losses
from sklearn import metrics
import random
import pickle

Using TensorFlow backend.


In [2]:
from os import walk
current_path = os.getcwd()
mypath = os.path.join(current_path, 'all-mias')
f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break

In [3]:
print(f)

['mdb001.pgm', 'mdb002.pgm', 'mdb003.pgm', 'mdb004.pgm', 'mdb005.pgm', 'mdb006.pgm', 'mdb007.pgm', 'mdb008.pgm', 'mdb009.pgm', 'mdb010.pgm', 'mdb011.pgm', 'mdb012.pgm', 'mdb013.pgm', 'mdb014.pgm', 'mdb015.pgm', 'mdb016.pgm', 'mdb017.pgm', 'mdb018.pgm', 'mdb019.pgm', 'mdb020.pgm', 'mdb021.pgm', 'mdb022.pgm', 'mdb023.pgm', 'mdb024.pgm', 'mdb025.pgm', 'mdb026.pgm', 'mdb027.pgm', 'mdb028.pgm', 'mdb029.pgm', 'mdb030.pgm', 'mdb031.pgm', 'mdb032.pgm', 'mdb033.pgm', 'mdb034.pgm', 'mdb035.pgm', 'mdb036.pgm', 'mdb037.pgm', 'mdb038.pgm', 'mdb039.pgm', 'mdb040.pgm', 'mdb041.pgm', 'mdb042.pgm', 'mdb043.pgm', 'mdb044.pgm', 'mdb045.pgm', 'mdb046.pgm', 'mdb047.pgm', 'mdb048.pgm', 'mdb049.pgm', 'mdb050.pgm', 'mdb051.pgm', 'mdb052.pgm', 'mdb053.pgm', 'mdb054.pgm', 'mdb055.pgm', 'mdb056.pgm', 'mdb057.pgm', 'mdb058.pgm', 'mdb059.pgm', 'mdb060.pgm', 'mdb061.pgm', 'mdb062.pgm', 'mdb063.pgm', 'mdb064.pgm', 'mdb065.pgm', 'mdb066.pgm', 'mdb067.pgm', 'mdb068.pgm', 'mdb069.pgm', 'mdb070.pgm', 'mdb071.pgm', 'mdb0

In [4]:
info=pd.read_csv("all-mias_info.txt",sep=" ")
info=info.drop('Unnamed: 7',axis=1)

In [5]:
info

,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS
0,mdb001,G,CIRC,B,535.0,425.0,197.0
1,mdb002,G,CIRC,B,522.0,280.0,69.0
2,mdb003,D,NORM,NaN,NaN,NaN,NaN
3,mdb004,D,NORM,NaN,NaN,NaN,NaN
4,mdb005,F,CIRC,B,477.0,133.0,30.0
...,...,...,...,...,...,...,...
325,mdb318,D,NORM,NaN,NaN,NaN,NaN
326,mdb319,D,NORM,NaN,NaN,NaN,NaN
327,mdb320,D,NORM,NaN,NaN,NaN,NaN
328,mdb321,D,NORM,NaN,NaN,NaN,NaN


In [6]:
info.dropna(subset = ["SEVERITY"], inplace=True)

info.reset_index(inplace = True)
info

,index,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS
0,0,mdb001,G,CIRC,B,535.0,425.0,197.0
1,1,mdb002,G,CIRC,B,522.0,280.0,69.0
2,4,mdb005,F,CIRC,B,477.0,133.0,30.0
3,5,mdb005,F,CIRC,B,500.0,168.0,26.0
4,10,mdb010,F,CIRC,B,525.0,425.0,33.0
...,...,...,...,...,...,...,...,...
118,281,mdb274,F,MISC,M,127.0,505.0,123.0
119,297,mdb290,D,CIRC,B,337.0,353.0,45.0
120,319,mdb312,F,MISC,B,240.0,263.0,20.0
121,321,mdb314,F,MISC,B,518.0,191.0,39.0


In [7]:
info = info.drop([3], axis=0)
info.reset_index(inplace = True)
info

,level_0,index,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS
0,0,0,mdb001,G,CIRC,B,535.0,425.0,197.0
1,1,1,mdb002,G,CIRC,B,522.0,280.0,69.0
2,2,4,mdb005,F,CIRC,B,477.0,133.0,30.0
3,4,10,mdb010,F,CIRC,B,525.0,425.0,33.0
4,5,12,mdb012,F,CIRC,B,471.0,458.0,40.0
...,...,...,...,...,...,...,...,...,...
117,118,281,mdb274,F,MISC,M,127.0,505.0,123.0
118,119,297,mdb290,D,CIRC,B,337.0,353.0,45.0
119,120,319,mdb312,F,MISC,B,240.0,263.0,20.0
120,121,321,mdb314,F,MISC,B,518.0,191.0,39.0


In [8]:
ids = {}
for i in range(len(info)):
    ids[i] = info.REFNUM[i]

In [9]:
label = []
for i in range(len(info)):
    if info.SEVERITY[i] == 'B':
        label.append(1)
    else:
        label.append(0)

In [10]:
label = np.array(label)

In [11]:
img_name = []

for i in range(len(label)):
        img_name.append(mypath +"\\" +info.REFNUM[i]+ '.pgm')

In [12]:
img_name = np.array(img_name)

In [13]:
img_path = []
last_label = []
for i in range(len(img_name)):
    
    img = cv2.imread(img_name[i], 0)
    img = cv2.resize(img, (224,224))
    rows, cols= img.shape
    for angle in range(360):
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)    #Rotate 0 degree
            img_rotated = cv2.warpAffine(img, M, (224, 224))
            img_path.append(img_rotated)
            if label[i] == 1:
                last_label.append(1)
            else:
                last_label.append(0)

In [14]:
last_label = np.array(last_label)
img_path = np.array(img_path)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(img_path, last_label, test_size = 0.2, random_state = 42)

In [16]:
len(x_train),len(x_test),len(y_train),len(y_test)

(35136, 8784, 35136, 8784)

In [17]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [18]:
x_train.shape

(35136, 224, 224)

In [19]:
(a,b,c)=x_train.shape # (35136, 224, 224)
x_train = np.reshape(x_train, (a, b, c, 1)) # 1 for gray scale
(a, b, c)=x_test.shape
x_test = np.reshape(x_test, (a, b, c, 1))

In [22]:
with open('bcc_cnn_224_360.pkl', 'rb') as file:
    pickle_model = pickle.load(file)

In [23]:
pickle_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 54, 54, 64)       

In [25]:
loss_value , accuracy = pickle_model.evaluate(x_test, y_test)

print('Test_loss_value = ' +str(loss_value))
print('test_accuracy = ' + str(accuracy))

print(pickle_model.predict(x_test))

8784/8784 [==============================] - 263s 30ms/step
Test_loss_value = 0.1851330772585122
test_accuracy = 0.9420537352561951
[[3.8691092e-01]
 [2.6667276e-01]
 [2.6099190e-06]
 ...
 [9.9920088e-01]
 [9.9450111e-01]
 [1.0639449e-01]]


In [27]:
y_pred=pickle_model.predict(x_test,verbose = 0)

In [30]:
#round_labels=np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, y_pred.round())
print(cm)

[[3617  248]
 [ 261 4658]]


In [31]:
precision = cm[1][1]/(cm[1][1] + cm[0][1])
recall = cm[1][1]/(cm[1][1] + cm[1][0])
f1 = 2*(precision * recall)/(precision + recall)
precision, recall, f1

(0.9494496534855279, 0.9469404350477739, 0.9481933842239186)